In [1]:
import torch
from torch import _utils
from torch.utils.data import DataLoader,Dataset,default_collate,RandomSampler,SequentialSampler,BatchSampler,Sampler
from torch import nn
import numpy as np

In [2]:
class TempDataset(Dataset):
    def __init__(self):
        super(TempDataset, self).__init__()
        pass
    def __getitem__(self,index):
        # print("getitem is called",index)
        return index*10
    def __len__(self):
        len=50
        # print("len is called ",len)
        return len

In [3]:
dataset = TempDataset()

In [4]:
import random

class Own_Random_sampler():
    def __init__(self,dataset):
        self.dataset = dataset
    def __iter__(self):
        list_indices=list(range(len(self.dataset)))
        # print(list_indices)
        random.shuffle(list_indices)
        return iter(list_indices)
    def __len__(self):
        return len(dataset)
       


In [5]:
class Own_Sequential_sampler():
    def __init__(self,dataset):
        self.dataset=dataset
    def __iter__(self):
        return iter(range(len(self.dataset)))
    def __len__(self):
        return len(dataset)

In [6]:
class Own_batch_sampler():
    def __init__(self,dataset=None,sampler=None,drop_last=0,batch_size=1):
        self.dataset = dataset
        self.drop_last = drop_last
        self.batch_size = batch_size
        self.sampler = sampler
        
    def __iter__(self):
        obj=iter(self.sampler)
        n=len(self.dataset)
        count=0

        global batched_indices
        batched_indices=[]
        

        for i in range(n//self.batch_size):
            batch=[]
            for i in range(self.batch_size):
                batch.append(dataset[next(obj)])
            count+=1
            batched_indices.append(batch)
            # print(batched_indices)

        if not self.drop_last:
            batch=[]
            for i in range(n-count*self.batch_size):
                batch.append(dataset[next(obj)])
            # print(batch)
            if len(batch)!=0:
                batched_indices.append(batch)
        return iter(batched_indices)
    def __len__(self):
        return len(batched_indices)

In [7]:
class OwnDataLoader:
    def __init__(self,dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, *, prefetch_factor=2,
           persistent_workers=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.sampler= sampler
        self.batch_sampler = batch_sampler
        self.drop_last = drop_last 
        # self.collate =  _utils.collate.default_collate

        
    
    def __iter__(self):
        
            
        if self.sampler:
            print("**")
            # s=self.sampler()
            # print(batches)
            if self.batch_size:
                batches=Own_batch_sampler(self.dataset,self.sampler,self.drop_last,self.batch_size)
                # batches=batch_sampler(self.dataset,self.sampler,self.drop_last,self.batch_size)
                return iter(batches)
            # if self.batch_size==1:

            
        if self.shuffle:
            # rand_obj =Own_Random_sampler(self.dataset)
            # batches=Own_batch_sampler(self.dataset,rand_obj,self.drop_last,self.batch_size)
            print("***")
            # print(batches)
            if self.batch_size:
                samp=Own_Random_sampler(self.dataset)
                batches=Own_batch_sampler(self.dataset,sampler=samp,drop_last=self.drop_last,batch_size=self.batch_size)
            return iter(batches)
            # return iter(list(range(0,len(self.dataset))))
        
        # if self.batch_size==1 and not self.sampler and not self.shuffle:
        #     samp=Own_Sequential_sampler(dataset)

        

        if (self.batch_size>1)or(self.batch_size==1 and not self.sampler and not self.shuffle and not self.batch_sampler):
            print("*")
            samp=Own_Sequential_sampler(dataset)
            batches=Own_batch_sampler(self.dataset,samp,self.drop_last,self.batch_size)
            return iter(batches)

        
        
        # if self.drop_last and self.batch_size==1:
        #     print("****")
        #     Rsamp=Own_Random_sampler(self.dataset)
        #     Ssamp=Own_Sequential_sampler(self.dataset)

        #     if not self.shuffle or self.sampler==None:
                
        #         batches=Own_batch_sampler(self.dataset,sampler=Ssamp,drop_last=self.drop_last,batch_size=self.batch_size)

        #     if self.shuffle or self.sampler != None:
        #         batches=Own_batch_sampler(self.dataset,sampler=Rsamp,drop_last=self.drop_last,batch_size=self.batch_size)
            
        #     return iter(batches)

        if self.batch_sampler:
            # for i in self.batch_sampler:
            #     print(i)
            if self.sampler is None and self.shuffle is False and self.batch_size==1:
                batches=iter(self.batch_sampler)
                return(batches)
            else :
                print("Sampler,Batch size ,shuffle is exclusive of Batch Sampler.")
    def __len__(self):
        return len(self.dataset)

In [8]:
random_sampler=Own_Random_sampler(dataset)
sequential_sampler=Own_Sequential_sampler(dataset)

batch_sampler=Own_batch_sampler(dataset,sequential_sampler,0,8)
for i in batch_sampler:
    print(i)

[0, 10, 20, 30, 40, 50, 60, 70]
[80, 90, 100, 110, 120, 130, 140, 150]
[160, 170, 180, 190, 200, 210, 220, 230]
[240, 250, 260, 270, 280, 290, 300, 310]
[320, 330, 340, 350, 360, 370, 380, 390]
[400, 410, 420, 430, 440, 450, 460, 470]
[480, 490]


In [9]:
def collate_func(batch):
    print(batch)

In [10]:
random_sampler=Own_Random_sampler(dataset)
for i in random_sampler:
    print(dataset[i],end=" ")

130 330 270 400 410 320 80 210 460 250 70 170 430 350 280 340 290 380 10 300 20 90 390 0 60 260 160 470 30 490 40 230 180 50 440 190 450 110 220 310 420 200 360 240 370 140 100 150 480 120 

In [11]:
random_sampler=Own_Sequential_sampler(dataset)
for i in random_sampler:
    print(dataset[i],end=" ")

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 

In [12]:
seq_sampler=Own_Sequential_sampler(dataset)
for i in seq_sampler:
    print(dataset[i],end=" ")

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 

In [13]:
# class OwnSeq_Sampler():
#     def __init__(dataset):
#         self.dataset = dataset
#     def __iter__():
#         return list(random.randint())


In [26]:
seq_sampler=Own_Sequential_sampler(dataset)
random_sampler=Own_Random_sampler(dataset)

# dataloader=OwnDataLoader(dataset,batch_size=3,sampler=seq_sampler,drop_last=1)
# dataloader=OwnDataLoader(dataset,sampler=random_sampler,batch_size=3,drop_last=1)
dataloader=OwnDataLoader(dataset,shuffle=True,batch_size=3,drop_last=1)
# dataloader=OwnDataLoader(dataset,batch_size=3)
# dataloader=OwnDataLoader(dataset,sampler=random_sampler,batch_size=3)
dataloader=OwnDataLoader(dataset)
# dataloader=OwnDataLoader(dataset,sampler=random_sampler,drop_last=False)



# batch_sampler=Own_batch_sampler(dataset=dataset,sampler=random_sampler,drop_last=True,batch_size=3)

# dataloader=OwnDataLoader(dataset,batch_sampler=batch_sampler)

print(len(dataloader))
# print(dataloader[i])
for i in dataloader:
    print(i)
    # print(i,end=" ")


50
*
[0]
[10]
[20]
[30]
[40]
[50]
[60]
[70]
[80]
[90]
[100]
[110]
[120]
[130]
[140]
[150]
[160]
[170]
[180]
[190]
[200]
[210]
[220]
[230]
[240]
[250]
[260]
[270]
[280]
[290]
[300]
[310]
[320]
[330]
[340]
[350]
[360]
[370]
[380]
[390]
[400]
[410]
[420]
[430]
[440]
[450]
[460]
[470]
[480]
[490]


In [15]:

# batch_sampler=Own_batch_sampler(dataset=dataset,sampler=random_sampler,drop_last=False,batch_size=2)
# dataloader=OwnDataLoader(dataset,batch_sampler=batch_sampler)
# for x,y in dataloader:
#     print(x,y)